In [16]:
a = [1, 2, 3, 4]

In [17]:
K.<a> = GF(2^8, modulus=[1,1,0,1,1,0,0,0,1])

In [18]:
print K.cardinality()

256


In [20]:
charpoly(a, 'x')

x^8 + x^4 + x^3 + x + 1

In [37]:
x = K..an_element()+2

In [42]:
print x+2

0


In [41]:
print x

0
